In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

import matplotlib.patches as mpatches

from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb

import os

In [2]:
def isIntersection(b1,b2):
    return not(b2[0] > b1[2] or b2[2] < b1[0] or b2[1] > b1[3] or b2[3] < b1[1])

In [3]:
def flex_resize(img_ori):
    MAX_AREA=img_ori.shape[0]*img_ori.shape[1]
    # decide the scale factor
    scale=1.0
    imgH=img_ori.shape[0]
    imgH_ref=img_ori.shape[0]
    while (imgH<160):
        scale+=0.1
        imgH=scale*imgH_ref
    # resize image
    dst=cv2.resize(img_ori,(0,0),fx= scale,fy= scale, interpolation=cv2.INTER_NEAREST)
    return dst,MAX_AREA

def getBoundingBox(dst,MAX_AREA):
    img_gray=cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
    img_median=cv2.medianBlur(img_gray,3)
    
    image_edges=cv2.Canny(img_median,100,200,3)#100,200,3

    kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    closing = cv2.morphologyEx(image_edges, cv2.MORPH_CLOSE, kernel)
    
    # clear the image border
    cleared = clear_border(closing)
    
    # label the image
    label_image = label(cleared)
    
    all_regions=regionprops(label_image)

    candidate_regions=[]
    valid_regions=[]
    isValid=0

    for region in all_regions:
        # take regions with large enough areas
        if region.area >= MAX_AREA/18:
            candidate_regions.append(region.bbox)

    for i in range(len(candidate_regions)):
        rr=candidate_regions[i]# reference box
        for j in range(len(candidate_regions)):
            if i==j:
                continue
            cr=candidate_regions[j]#the box to be compared with
            if cr[0]<rr[0] and cr[1]<rr[1] and cr[2]>rr[2] and cr[3]>rr[3]:
                isValid=0
                break
            else:
                isValid=1
        if isValid:
            valid_regions.append(rr)
            
    return valid_regions

In [4]:
all_files=[]
root_dir="/home/fengsibo/my_project/OCR/image_segmentation/capture/"
for parent,dirnames,filenames in os.walk(root_dir):
       for filename in filenames:
            full_filenames=filename
            all_files.append(full_filenames)
            #print "filename is:" + full_filenames
len(all_files)
#all_files[1][:-4]

840

In [5]:
result_dir="/home/fengsibo/my_project/OCR/image_segmentation/result"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)
    
v_r=[]
for file_id in range(len(all_files)):
    filename=root_dir+all_files[file_id]
    img_ori=cv2.imread(filename,1)
    dst,MAX_AREA=flex_resize(img_ori)
    v_r=getBoundingBox(dst,MAX_AREA)#valid region
    for i in range(len(v_r)):
        img=dst[v_r[i][0]:v_r[i][2],v_r[i][1]:v_r[i][3]]
        result_name=result_dir+'/'+all_files[file_id][:-4]+'_'+str(i)+'.jpg'
        cv2.imwrite(result_name,img)

In [6]:
'''
image_label_overlay = label2rgb(label_image, image=dst)

fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)
for region in valid_regions:
    # take regions with large enough areas
    minr, minc, maxr, maxc = region
    #print region.bbox
    rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                            fill=False, edgecolor='red', linewidth=2)
    ax.add_patch(rect)
        #print type(region.bbox)
ax.set_axis_off()
plt.tight_layout()
plt.show()
'''

"\nimage_label_overlay = label2rgb(label_image, image=dst)\n\nfig, ax = plt.subplots(figsize=(10, 6))\nax.imshow(image_label_overlay)\nfor region in valid_regions:\n    # take regions with large enough areas\n    minr, minc, maxr, maxc = region\n    #print region.bbox\n    rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,\n                            fill=False, edgecolor='red', linewidth=2)\n    ax.add_patch(rect)\n        #print type(region.bbox)\nax.set_axis_off()\nplt.tight_layout()\nplt.show()\n"